# Работа

In [1]:
import pandas as pd
import numpy as np

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")

%load_ext lab_black

## 1. Подготовка файла

In [2]:
%%time

# вспомогательные функции

def oktmo(o):
    """
    функция дополняет код ОКТМО до 11 символов или до 10 символов без ведущего 0
    """
    if len(str(o)) == 1:
        l = 9
    else:
        l = 11-len(str(o))
    return int(str(o) + "0"*(l))
    
def visit_count(row):
    """
    функция определяет количество посещений согласно условиям
    """
    if row['purpose'] < 3:
        return row['sl_id']
    elif row['purpose'] >= 3:
        res = row['code_usl'] * row['kol_usl']
        try:
            if np.isnan(res):
                return 0
        except:
            return res
    else:
        return 'не определено'
    
# загрузка справочников
f_032 = pd.read_excel("/home/sk27/work/med/data/F032.xlsx")
v_021 = pd.read_excel("/home/sk27/work/med/data/V021.xlsx")
v_002 = pd.read_excel("/home/sk27/work/med/data/V002.xlsx")
v_006 = pd.read_excel("/home/sk27/work/med/data/V006.xlsx")
v_025 = pd.read_excel("/home/sk27/work/med/data/V025.xlsx")
oid_data = pd.read_excel("/home/sk27/work/med/data/oid_data.xlsx")

# актуализация справочников
v_021 = v_021[v_021["DATEEND"].isna()]
v_002 = v_002[v_002["DATEEND"].isna()]
v_006 = v_006[v_006["DATEEND"].isna()]
v_025 = v_025[v_025["DATEEND"].isna()]


# добавление в файл данных строки заголовков
with open("/home/sk27/work/med/data/source.csv", "r") as f:
    first_line = next(f).strip()
    if first_line[0:4] != "year":
        with open(
            "/home/sk27/work/med/data/source.csv", "r", encoding="ISO-8859-1"
        ) as original:
            data = original.read()
        with open("/home/sk27/work/med/data/source.csv", "w") as modified:
            modified.write(
                "year;month;region;clinic;oid;depart;branch;doctor_specialty;doctor_profile;terms;doctor_id;diagnosis;purpose;service_id;usl_service_id;usl_doctor_specialty;usl_clinic;usl_depart;sl_id;code_usl;kol_usl;yet_sum\n"
                + data
            )
    else:
        pass

# загрузка файла
df_data = pd.read_csv(
    "/home/sk27/work/med/data/source.csv",
    sep=";",
    encoding="ISO-8859-1",
    low_memory=False,
)

# удаление кавычек
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace('"', "")
# df_data = df_data.apply(pd.to_numeric, errors="ignore")

# замена ВСЕХ запятых точками
for i, col in enumerate(df_data.columns):
    df_data.iloc[:, i] = df_data.iloc[:, i].astype("str").str.replace(',', ".")
df_data = df_data.apply(pd.to_numeric, errors="ignore")

# вставка столбца ОКТМО
df_data.insert(3, "oktmo", df_data["region"].apply(oktmo))

# проверка загруженных данных

# а) проверка количества столбцов - их должно быть именно 23 вместе с добавленным мной столбцом ОКТМО
if df_data.shape[1] > 23:
    print(f"Количество полей в файле (их {df_data.shape[1]}) не соответствует форме отчёта, должно быть 23.")
else:
    pass

# б) проверка столбца terms на "только 3"
try:
    if df_data["terms"].mean() != 3:
        print(
            "Нарушено требование об указании в поле 'Условия оказания медпомощи' только значения '3'."
        )
except:
    print("Не проходит проверка на '3'")
    
# расшифровка кодов

CPU times: user 53.1 s, sys: 421 ms, total: 53.6 s
Wall time: 53.6 s


In [3]:
df_data.head(2)

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2022,9,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
1,2022,9,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,3.0,B01.023.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0


In [ ]:
# расшифровка субъекта и МО
df_data = df_data.merge(f_032, left_on='region', right_on='MCOD', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка медучреждения
df_data = df_data.merge(oid_data, left_on='oid', right_on='oid', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка специальности врача
df_data = df_data.merge(v_021, left_on='doctor_specialty', right_on='IDSPEC', how='left')
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on='doctor_profile', right_on='IDPR', how='left')
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on='terms', right_on='IDUMP', how='left')
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on='purpose', right_on='IDPC', how='left')

### Расшифровка кодов

In [8]:
v_021

,IDSPEC,SPECNAME,POSTNAME,IDPOST_MZ,DATEBEG,DATEEND
1,1,Авиационная и космическая медицина,врач по авиационной и космической медицине,62.0,2019-11-01,NaT
3,2,Акушерство и гинекология,врач-акушер-гинеколог,13.0,2019-11-01,NaT
5,3,Аллергология и иммунология,врач-аллерголог-иммунолог,15.0,2019-11-01,NaT
7,4,Анестезиология-реаниматология,врач-анестезиолог-реаниматолог,16.0,2019-11-01,NaT
9,5,Бактериология,врач-бактериолог,17.0,2019-11-01,NaT
...,...,...,...,...,...,...
228,231,Диетология,медицинская сестра диетическая,162.0,2019-11-01,NaT
230,233,Стоматология профилактическая,гигиенист стоматологический,151.0,2019-11-01,NaT
233,280,Наркология,фельдшер-нарколог,197.0,2019-11-01,NaT
235,281,Реабилитационное сестринское дело,медицинская сестра по реабилитации,172.0,2019-11-01,NaT


In [4]:
# расшифорвка названия субьекта Росссийской Федерации
df_data = df_data.merge(f_032, left_on="oktmo", right_on="OKTMO_P", how="left")

In [5]:
# расшифровка наименования (сокращённого) медучреждения
df_data = df_data.merge(oid_data, left_on="oid", right_on="oid", how="left")

In [6]:
# расшифровка специальности врача
df_data = df_data.merge(
    v_021, left_on="doctor_specialty", right_on="IDSPEC", how="left"
)

In [7]:
df_data.columns

Index(['year', 'month', 'region', 'oktmo', 'clinic', 'oid', 'depart', 'branch',
       'doctor_specialty', 'doctor_profile', 'terms', 'doctor_id', 'diagnosis',
       'purpose', 'service_id', 'usl_service_id', 'usl_doctor_specialty',
       'usl_clinic', 'usl_depart', 'sl_id', 'code_usl', 'kol_usl', 'yet_sum',
       'UIDMO', 'IDMO', 'MCOD', 'OKTMO_P', 'SUBJ', 'D_BEGIN', 'D_END',
       'NAME_E', 'DATEBEG_x', 'DATEEND_x', 'id', 'oldOid', 'nameFull',
       'nameShort', 'parentId', 'medicalSubjectId', 'medicalSubjectName',
       'inn', 'kpp', 'ogrn', 'regionId', 'regionName', 'organizationType',
       'moDeptId', 'moDeptName', 'founder', 'deleteDate', 'deleteReason',
       'createDate', 'modifyDate', 'moLevel', 'moAgencyKindId', 'moAgencyKind',
       'profileAgencyKindId', 'profileAgencyKind', 'postIndex',
       'cadastralNumber', 'latitude', 'longtitude', 'fiasVersion', 'aoidArea',
       'aoidStreet', 'houseid', 'addrRegionId', 'addrRegionName',
       'territoryCode', 'isFederal

In [9]:
df_data_result = (
    df_data[
        [
            "year",
            "month",
            "region",
            "oktmo",
            "SUBJ",
            "clinic",
            "oid",
            "nameShort",
            "depart",
            "branch",
            "doctor_specialty",
            "POSTNAME",
            "doctor_profile",
            "terms",
            "doctor_id",
            "diagnosis",
            "purpose",
            "service_id",
            "usl_service_id",
            "usl_doctor_specialty",
            "usl_clinic",
            "usl_depart",
            "sl_id",
            "code_usl",
            "kol_usl",
            "yet_sum",
        ]
    ]
    .copy()
    .reset_index(drop=True)
)

In [10]:
df_data_result

,year,month,region,oktmo,SUBJ,clinic,oid,nameShort,depart,branch,...,purpose,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum
0,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
1,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
2,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
3,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
4,2022,9,61000,61000000000,Рязанская область,620077,1.2.643.5.1.13.13.12.2.62.6355,ГБУ РО «ГОРОДСКАЯ ДЕТСКАЯ ПОЛИКЛИНИКА № 3»,073_2,6.0,...,3.0,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5265458,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,2.6,A03.16.001,068-363-489 00,93,620105,291_2,1,1,1.0,0.0
5265459,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,2.6,A03.16.001,068-363-489 00,93,620105,291_2,1,1,1.0,0.0
5265460,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,2.6,A03.16.001,068-363-489 00,93,620105,291_2,1,1,1.0,0.0
5265461,2023,2,61000,61000000000,Рязанская область,620105,1.2.643.5.1.13.13.12.2.62.6379,"ГБУ РО ""ГКБСМП""",291_2,10.0,...,2.6,A03.16.001,068-363-489 00,93,620105,291_2,1,1,1.0,0.0


## 2. Удаление дубликатов

In [6]:
%%time

# вспомогательная функция: определение количества посещений
def visit_count(row):
    '''функция определяет количество посещений согласно условиям'''
    if row['purpose'] < 3:
        return row['sl_id']
    elif row['purpose'] >= 3:
        res = row['code_usl'] * row['kol_usl']
        try:
            if np.isnan(res):
                return 0
        except:
            return res
    else:
        return 'не определено'

# расшифровка идентификаторов

# загрузка и очистка данных
df_data = pd.read_csv('C:/Users/Admin/Documents/work/data/source.csv', sep=';', encoding = 'ISO-8859-1', low_memory=False)
df_data = df_data.apply(pd.to_numeric, errors='ignore')
df_data['purpose'] = df_data['purpose'].apply(pd.to_numeric, errors='coerce')
# сохранение дубликатов на диск
df_data_duplicates = df_data[df_data.duplicated()].reset_index()
df_data_duplicates.to_csv('C:/Users/Admin/Documents/work/data/duplicates.csv')
# удаление дубликатов
df_data = df_data.drop_duplicates().reset_index(drop=True)
# загрузка справочников
f_032 = pd.read_excel("C:/Users/Admin/Documents/work/data/F032.xlsx")
v_021 = pd.read_excel("C:/Users/Admin/Documents/work/data/V021.xlsx")
v_002 = pd.read_excel("C:/Users/Admin/Documents/work/data/V002.xlsx")
v_006 = pd.read_excel("C:/Users/Admin/Documents/work/data/V006.xlsx")
v_025 = pd.read_excel("C:/Users/Admin/Documents/work/data/V025.xlsx")
oid_data = pd.read_excel("C:/Users/Admin/Documents/work/data/oid_data.xlsx")
# расшифровка субъекта и МО
df_data = df_data.merge(f_032, left_on='region', right_on='MCOD', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка медучреждения
df_data = df_data.merge(oid_data, left_on='oid', right_on='oid', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка специальности врача
df_data = df_data.merge(v_021, left_on='doctor_specialty', right_on='IDSPEC', how='left')
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on='doctor_profile', right_on='IDPR', how='left')
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on='terms', right_on='IDUMP', how='left')
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on='purpose', right_on='IDPC', how='left')

CPU times: total: 1min 43s
Wall time: 1min 45s


In [4]:
# фильтрация данных
df_data = df_data[['year', 'month', 'region', 'SUBJ_x', 'clinic', 'SUBJ_y', 'oid', 'nameShort', 'depart', 'branch',
       'doctor_specialty', 'SPECNAME', 'doctor_profile', 'PRNAME', 'terms', 'IDUMP', 'doctor_id', 'diagnosis',
       'purpose', 'N_PC', 'service_id', 'usl_service_id', 'usl_doctor_specialty',
       'usl_clinic', 'usl_depart', 'sl_id', 'code_usl', 'kol_usl', 'yet_sum']].copy().drop_duplicates().reset_index(drop=True)
# определение количества посещений
df_data['visit_count'] = df_data.apply(visit_count, axis=1)

In [5]:
df_data

,year,month,region,SUBJ_x,clinic,SUBJ_y,oid,nameShort,depart,branch,...,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,visit_count
0,2023,1,25000,Республика Башкортостан,380049,Иркутская область,1.2.643.5.1.13.13.12.2.38.3568,"ОГБУЗ ""ИГП №15""",1,1,...,B01.047.001,186,1,1,1.0,0.00,NaN,NaN,NaN,0
1,2023,1,25000,Республика Башкортостан,380049,Иркутская область,1.2.643.5.1.13.13.12.2.38.3568,"ОГБУЗ ""ИГП №15""",1,1,...,B01.047.001,035,1,1,1.0,0.00,NaN,NaN,NaN,0
2,2023,1,25000,Республика Башкортостан,380049,Иркутская область,1.2.643.5.1.13.13.12.2.38.3568,"ОГБУЗ ""ИГП №15""",1,1,...,B01.047.001,192,1,1,1.0,0.00,NaN,NaN,NaN,0
3,2023,1,25000,Республика Башкортостан,380049,Иркутская область,1.2.643.5.1.13.13.12.2.38.3568,"ОГБУЗ ""ИГП №15""",2,2,...,B01.026.001.6,162,1,1,1.0,0.00,NaN,NaN,NaN,0
4,2023,1,25000,Республика Башкортостан,380049,Иркутская область,1.2.643.5.1.13.13.12.2.38.3568,"ОГБУЗ ""ИГП №15""",1,1,...,B01.047.002,203,3,1,3.0,0.00,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517113,2023,2,25000,Республика Башкортостан,380038,Иркутская область,1.2.643.5.1.13.13.12.2.38.3528,"ОГАУЗ ""ИСП № 1""",4,4,...,B01.065.001,260,19,8,60.0,28.44,NaN,NaN,NaN,28.44
517114,2023,2,25000,Республика Башкортостан,380086,Иркутская область,1.2.643.5.1.13.13.12.2.38.9663,ФГБОУ ВО ИГМУ Минздрава России,4,4,...,B01.067.001,043,1,1,1.0,1.40,NaN,NaN,NaN,0
517115,2023,1,25000,Республика Башкортостан,380029,Иркутская область,1.2.643.5.1.13.13.12.2.38.3558,ОГАУЗ ИГКБ № 8,4,4,...,B01.065.001,599,2,2,2.0,3.63,NaN,NaN,NaN,0
517116,2023,2,25000,Республика Башкортостан,380021,Иркутская область,1.2.643.5.1.13.13.12.2.38.3547,"ОГАУЗ ""МЕДСАНЧАСТЬ ИАПО""",4,4,...,B01.064.003,505,4,4,6.0,4.58,NaN,NaN,NaN,0


In [30]:
# сохранение результатов на диск
df_data.to_csv('C:/Users/Admin/Documents/work/data/result.csv', encoding = 'utf-8-sig')

In [8]:
f_032 = pd.read_excel("C:/Users/Admin/Documents/work/data/F032.xlsx")
f_032

,UIDMO,IDMO,MCOD,OKTMO_P,SUBJ,D_BEGIN,D_END,NAME_E,DATEBEG,DATEEND
0,1202201101000000,3503000000007,10078,79000000000,Республика Адыгея,2021-08-19,NaT,NaN,2022-01-01,2022-10-11
1,1202201102000000,3503000000007,10078,79000000000,Республика Адыгея,2022-10-11,NaT,NaN,2022-10-12,2022-12-15
2,1202300100000000,29300000000000,10076,79000000000,Республика Адыгея,2022-06-03,NaT,NaN,2023-01-01,2023-12-31
3,1202300200000000,70800000000000,10079,79000000000,Республика Адыгея,2022-07-04,NaT,NaN,2023-01-01,2023-12-31
4,1202300300000000,1235000000000000,10049,79000000000,Республика Адыгея,2022-07-05,NaT,NaN,2023-02-15,2023-12-31
...,...,...,...,...,...,...,...,...,...,...
14512,99202337900000000,147300000000000,990186,0,Российская Федерация,2022-08-31,NaT,NaN,2023-01-01,2023-12-31
14513,99202338000000000,118700000000000,990170,0,Российская Федерация,2022-08-31,NaT,NaN,2023-01-01,2023-12-31
14514,99202338100000000,404500000000000,990340,0,Российская Федерация,2022-09-05,NaT,NaN,2023-01-01,2023-12-31
14515,99202338200000000,933700000000000,990113,0,Российская Федерация,2022-11-03,NaT,NaN,2023-01-01,2023-12-31


In [ ]:
df_data['state'].apply(lambda x: x[len(x)/2-1:len(x)/2+1])

## 2. Без удаления дубликатов (НЕ СДЕЛАНО!!!)

In [3]:
%%time

# вспомогательная функция: определение количества посещений
def visit_count(row):
    '''функция определяет количество посещений согласно условиям'''
    if row['purpose'] < 3:
        return row['sl_id']
    elif row['purpose'] >= 3:
        res = row['code_usl'] * row['kol_usl']
        try:
            if np.isnan(res):
                return 0
        except:
            return res
    else:
        return 'не определено'

# расшифровка идентификаторов

# загрузка и очистка данных
df_data = pd.read_csv('C:/Users/Admin/Documents/work/data/25000-POS-D1.csv', sep=';', encoding = 'ISO-8859-1', low_memory=False)
df_data = df_data.apply(pd.to_numeric, errors='ignore')
df_data['purpose'] = df_data['purpose'].apply(pd.to_numeric, errors='coerce')
# сохранение дубликатов на диск
df_data_duplicates = df_data[df_data.duplicated()].reset_index()
df_data_duplicates.to_csv('C:/Users/Admin/Documents/work/data/25000-POS-D1_duplicates.csv')
# удаление дубликатов
# df_data = df_data.drop_duplicates().reset_index(drop=True)
# загрузка справочников
f_032 = pd.read_excel("C:/Users/Admin/Documents/work/data/F032.xlsx")
v_021 = pd.read_excel("C:/Users/Admin/Documents/work/data/V021.xlsx")
v_002 = pd.read_excel("C:/Users/Admin/Documents/work/data/V002.xlsx")
v_006 = pd.read_excel("C:/Users/Admin/Documents/work/data/V006.xlsx")
v_025 = pd.read_excel("C:/Users/Admin/Documents/work/data/V025.xlsx")
oid_data = pd.read_excel("C:/Users/Admin/Documents/work/data/oid_data.xlsx")
# расшифровка субъекта и МО
df_data = df_data.merge(f_032, left_on='region', right_on='MCOD', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка медучреждения
df_data = df_data.merge(oid_data, left_on='oid', right_on='oid', how='left').merge(f_032, left_on="clinic", right_on='MCOD', how='left').reset_index(drop=True)
# расшифровка специальности врача
df_data = df_data.merge(v_021, left_on='doctor_specialty', right_on='IDSPEC', how='left')
# расшифровка профиля врача
df_data = df_data.merge(v_002, left_on='doctor_profile', right_on='IDPR', how='left')
# расшифровка условий оказания помощи
df_data = df_data.merge(v_006, left_on='terms', right_on='IDUMP', how='left')
# расшифровка цели посещения
df_data = df_data.merge(v_025, left_on='purpose', right_on='IDPC', how='left')
# фильтрация данных
df_data = df_data[['year', 'month', 'region', 'SUBJ_x', 'clinic', 'SUBJ_y', 'oid', 'nameShort', 'depart', 'branch',
       'doctor_specialty', 'SPECNAME', 'doctor_profile', 'PRNAME', 'terms', 'IDUMP', 'doctor_id', 'diagnosis',
       'purpose', 'N_PC', 'service_id', 'usl_service_id', 'usl_doctor_specialty',
       'usl_clinic', 'usl_depart', 'sl_id', 'code_usl', 'kol_usl', 'yet_sum']].copy().reset_index(drop=True)
# определение количества посещений
df_data['visit_count'] = df_data.apply(visit_count, axis=1)

KeyboardInterrupt: 

In [4]:
df_data

,year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,...,D_END,NAME_E,DATEBEG_x,DATEEND_x,IDSPEC,SPECNAME,POSTNAME,IDPOST_MZ,DATEBEG_y,DATEEND_y
0,2023,1,25000,380049,1.2.643.5.1.13.13.12.2.38.3568,1,1,76,97,3,...,NaT,NaN,2023-01-01,2023-12-31,76,Терапия,NaN,NaN,2015-11-28,2019-10-31
1,2023,1,25000,380049,1.2.643.5.1.13.13.12.2.38.3568,1,1,76,97,3,...,NaT,NaN,2023-01-01,2023-12-31,76,Терапия,врач-терапевт,109.0,2019-11-01,NaT
2,2023,1,25000,380049,1.2.643.5.1.13.13.12.2.38.3568,1,1,76,97,3,...,NaT,NaN,2023-01-01,2023-12-31,76,Терапия,NaN,NaN,2015-11-28,2019-10-31
3,2023,1,25000,380049,1.2.643.5.1.13.13.12.2.38.3568,1,1,76,97,3,...,NaT,NaN,2023-01-01,2023-12-31,76,Терапия,врач-терапевт,109.0,2019-11-01,NaT
4,2023,1,25000,380049,1.2.643.5.1.13.13.12.2.38.3568,1,1,76,97,3,...,NaT,NaN,2023-01-01,2023-12-31,76,Терапия,NaN,NaN,2015-11-28,2019-10-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170581,2023,2,25000,380021,1.2.643.5.1.13.13.12.2.38.3547,4,4,71,86,3,...,NaT,NaN,2023-01-01,2023-12-31,71,Стоматология терапевтическая,врач-стоматолог-терапевт,103.0,2019-11-01,NaT
4170582,2023,2,25000,380059,1.2.643.5.1.13.13.12.2.38.3581,4,4,72,90,3,...,NaT,NaN,2023-01-01,2023-12-31,72,Стоматология хирургическая,NaN,NaN,2015-11-28,2019-10-31
4170583,2023,2,25000,380059,1.2.643.5.1.13.13.12.2.38.3581,4,4,72,90,3,...,NaT,NaN,2023-01-01,2023-12-31,72,Стоматология хирургическая,врач-стоматолог-хирург,104.0,2019-11-01,NaT
4170584,2023,2,25000,380139,1.2.643.5.1.13.13.12.2.38.3595,4,4,68,86,3,...,NaT,NaN,2023-03-11,2023-12-31,68,Стоматология детская,NaN,NaN,2015-11-28,2019-10-31


In [21]:
df_data = pd.read_csv('C:/Users/Admin/Documents/work/data/25000-POS-D1.csv', sep=';', encoding = 'ISO-8859-1', low_memory=False)

In [22]:
df_data.replace(['"'], [''], inplace=True)

In [27]:
df_data

,ï»¿year,month,region,clinic,oid,depart,branch,doctor_specialty,doctor_profile,terms,...,DATEBEG_x,DATEEND_x,IDUMP,UMPNAME,DATEBEG_y,DATEEND_y,IDPC,N_PC,DATEBEG,DATEEND
0,2022,9,61000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,3,...,2013-07-08,NaT,3,Амбулаторно,2012-10-18,NaN,3.0,Обращение по заболеванию,2018-01-01,NaN
1,2022,9,61000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,3,...,2013-07-08,NaT,3,Амбулаторно,2012-10-18,NaN,3.0,Обращение по заболеванию,2018-01-01,NaN
2,2022,9,61000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,3,...,2013-07-08,NaT,3,Амбулаторно,2012-10-18,NaN,3.0,Обращение по заболеванию,2018-01-01,NaN
3,2022,9,61000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,3,...,2013-07-08,NaT,3,Амбулаторно,2012-10-18,NaN,3.0,Обращение по заболеванию,2018-01-01,NaN
4,2022,10,61000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,3,...,2013-07-08,NaT,3,Амбулаторно,2012-10-18,NaN,3.0,Обращение по заболеванию,2018-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102103,2023,2,61000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,3,...,2011-01-01,NaT,3,Амбулаторно,2012-10-18,NaN,2.6,Посещение по другим обстоятельствам,2018-01-01,NaN
102104,2023,2,61000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,3,...,2011-01-01,NaT,3,Амбулаторно,2012-10-18,NaN,2.6,Посещение по другим обстоятельствам,2018-01-01,NaN
102105,2023,2,61000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,3,...,2011-01-01,NaT,3,Амбулаторно,2012-10-18,NaN,2.6,Посещение по другим обстоятельствам,2018-01-01,NaN
102106,2023,2,61000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,3,...,2011-01-01,NaT,3,Амбулаторно,2012-10-18,NaN,2.6,Посещение по другим обстоятельствам,2018-01-01,NaN


In [18]:
# нужно получить 1000000
lst = [1, 10, 100, 1000]
for element in lst:
    print(len(str(element)))
    print(7 - len(str(element)))
    print("0" * (7 - len(str(element))))
    print(str(element) + "0" * (7 - len(str(element))))

1
6
000000
1000000
2
5
00000
1000000
3
4
0000
1000000
4
3
000
1000000


In [36]:
def oktmo_test(row):
    """
    функция дополняет код ОКТМО до 11 символов или до 10 символов без ведущего 0
    """
    if len(str(row)) == 1:
        l = 9
    else:
        l = 11-len(str(row))
    return int(str(row) + "0"*(l))

In [38]:
oktmo_test(65000)

65000000000

In [33]:
df_data["test"] = df_data["region"].apply(oktmo_test)

In [34]:
df_data

,year,month,region,oktmo,clinic,oid,depart,branch,doctor_specialty,doctor_profile,...,service_id,usl_service_id,usl_doctor_specialty,usl_clinic,usl_depart,sl_id,code_usl,kol_usl,yet_sum,test
0,2022,9,1,10000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,B01.020.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0,10000000000
1,2022,9,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,B01.023.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0,61000000000
2,2022,10,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,B01.010.001,030-646-635 32,35,620077,073_2,1,1,1.0,0.0,61000000000
3,2022,10,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,B01.020.005,030-646-635 32,35,620077,073_2,1,1,1.0,0.0,61000000000
4,2022,10,61000,61000000000,620077,1.2.643.5.1.13.13.12.2.62.6355,073_2,6.0,35,158,...,B01.023.002,030-646-635 32,35,620077,073_2,1,1,1.0,0.0,61000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51116,2023,2,61000,61000000000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,81,106,...,A04.12.006.001,031-663-288 38,81,620105,291_2,4,4,4.0,0.0,61000000000
51117,2023,2,61000,61000000000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,81,106,...,A04.12.006.002,031-663-288 38,81,620105,291_2,26,26,26.0,0.0,61000000000
51118,2023,2,61000,61000000000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,...,A03.16.001,068-363-489 00,93,620105,291_2,9,9,9.0,0.0,61000000000
51119,2023,2,61000,61000000000,620105,1.2.643.5.1.13.13.12.2.62.6379,291_2,10.0,93,123,...,A03.18.001.001,068-363-489 00,93,620105,291_2,11,11,11.0,0.0,61000000000
